In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


In [2]:
url_data = "https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=BlocksFavour&agg=Accumulated"
years = list(range(2001, 2022))

In [3]:

for year in years:
    with open("blocks_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [4]:
dfs = []
for year in years :
    with open ("blocks_leaders_htmls/{}.html".format(year),encoding="utf-8") as f :
        page =f.read()
        soup = BeautifulSoup(page, "html.parser")
        soup.find('tr',class_="StatsHeaderGridResults")
        statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")
        blocks_data= pd.read_html(str(statistics_table))[0]
        blocks_data["Year"] = year
        dfs.append(blocks_data)
                

In [5]:
blocks_data = pd.concat(dfs)
blocks_data

,Rank,Player,Team,GamesG,BlocksBL+,Ave.,Per 40 min.40m,Year
0,1,"KHIZHNIAK, GRYGORIIKHIZHNIAK, G.",ZALZalgirisZalgiris,14,45,3.21,4.00,2001
1,2,"MIKHAILOV, MIKHAILMIKHAILOV, M.",PEMUral GreatUral Great,19,31,1.63,2.56,2001
2,3,"STRUELENS, ERICSTRUELENS, E.",RMDReal MadridReal Madrid,16,23,1.44,2.52,2001
3,4,"SAVRASENKO, ALEKSEYSAVRASENKO, A.",OLYOlympiacosOlympiacos,20,22,1.10,2.08,2001
4,5,"GOREE, MARCUSGOREE, M.",FRAOpel SkylinersOpel Skyliners,14,21,1.50,1.70,2001
...,...,...,...,...,...,...,...,...
45,44,"WHITE, OKAROWHITE, O.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,29,9,0.31,0.54,2021
46,44,"VOIGTMANN, JOHANNESVOIGTMANN, J.",CSKCSKA MoscowCSKA Moscow,20,9,0.45,0.85,2021
47,44,"REYNOLDS, JALENREYNOLDS, J.",MTAMaccabi Playtika Tel AvivMaccabi Playtika T...,35,9,0.26,0.62,2021
48,44,"FLOYD, JEHYVEFLOYD, J.",2 PAOPanathinaikos OPAP AthensPanathinaikos OP...,24,9,0.38,1.25,2021


In [9]:
engine = create_engine('sqlite:///blocks_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
blocks_data.to_sql(
    name = "blocks_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [11]:
blocks_data_select = session.execute('select * from "blocks_leaders"')
blocks_data_select.all()

[(0, 1, 'DUNSTON, BRYANTDUNSTON, B.', 'EFSAnadolu Efes IstanbulAnadolu Efes Istanbul', 30, 52, 1.73, 2.46, 2001),
 (1, 2, 'TAVARES, WALTERTAVARES, W.', 'RMBReal MadridReal Madrid', 29, 43, 1.48, 3.4, 2001),
 (2, 3, 'PLEISS, TIBORPLEISS, T.', 'VBCValencia BasketValencia Basket', 29, 32, 1.1, 2.29, 2001),
 (3, 4, 'PARAKHOUSKI, ARTSIOMPARAKHOUSKI, A.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 30, 31, 1.03, 2.55, 2001),
 (4, 4, 'GUDAITIS, ARTURASGUDAITIS, A.', 'AXMAX Armani Exchange Olimpia MilanAX Armani Exchange Olimpia Milan', 29, 31, 1.07, 2.07, 2001),
 (5, 6, 'TYUS, ALEXTYUS, A.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 30, 30, 1.0, 2.0, 2001),
 (6, 7, 'POIRIER, VINCENTPOIRIER, V.', 'BKNKIROLBET Baskonia Vitoria GasteizKIROLBET Baskonia Vitoria Gasteiz', 34, 29, 0.85, 1.91, 2001),
 (7, 8, 'BOLDEN, JONAHBOLDEN, J.', 'MTAMaccabi FOX Tel AvivMaccabi FOX Tel Aviv', 29, 27, 0.93, 1.76, 2001),
 (8, 9, 'SINGLETON, CHRISSINGLETON, C.', 'PAOPanathinaikos Superfoods AthensPanathina